In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd

ruta = '/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Tabla_Clasificada_por_Cuadrante.csv'
df = pd.read_csv(ruta, encoding='latin1')  # o encoding='ISO-8859-1'
df.head()


,Iniciativa,Impacto,Alineacion,ROI,Escalabilidad,Replicabilidad,Madurez,Tiempo,Riesgo,Complejidad,Beneficio promedio,Area,Tiempo.1,Costo desarrollo,Costo renta,Beneficio (%),Indice Financiero Total (%),Categoria
0,Gestor de Propuestas Tacnicas,5,4,3,5,4,3,5,5,5,4.33,SMC,8.50,"$4,620.00",$84.24,86.6,24.21,Alta prioridad
1,Propuestas Servicio Base,5,4,3,4,4,3,5,5,5,4.22,CSP,29.75,"$14,236.00",$874.59,84.4,77.76,Media prioridad
2,Seguimiento Tickets SOC,5,4,3,4,4,3,5,5,5,4.22,MDR SOC,23.50,"$12,170.00","$1,530.62",84.4,70.51,Media prioridad
3,Candidato 360,4,5,3,4,4,3,5,5,5,4.22,RRHH,14.25,"$7,429.00",$15.15,84.4,38.31,Alta prioridad
4,Automatizacion Precalificacion CRM,5,4,3,5,3,3,5,5,5,4.22,Ventas,20.25,"$10,340.00","$8,470.96",84.4,96.80,Media prioridad


In [7]:
# Imprime los nombres tal cual
print(df.columns.tolist())

# Usa el nombre correcto detectado, por ejemplo:
# Si ves que es 'Beneficio (%)' o 'Beneficio promedio' pon ese


['Iniciativa', 'Impacto', 'Alineacion', 'ROI', 'Escalabilidad', 'Replicabilidad', 'Madurez', 'Tiempo', 'Riesgo', 'Complejidad', 'Beneficio promedio', 'Area', 'Tiempo.1', 'Costo desarrollo', 'Costo renta', 'Beneficio (%)', 'Indice Financiero Total (%)', 'Categoria']


In [16]:
import pandas as pd
import plotly.graph_objects as go

# 1. Cargar el CSV
ruta = '/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Tabla_Clasificada_por_Cuadrante.csv'
df = pd.read_csv(ruta, encoding='latin1')

# 2. Clasificación automática para el gráfico (puedes omitir si ya viene en CSV)
def clasificar_prioridad(beneficio, costo):
    if beneficio >= 50 and costo <= 50:
        return 'Alta Prioridad'
    elif beneficio >= 50 and costo > 50:
        return 'Media Prioridad'
    elif beneficio < 50 and costo <= 50:
        return 'Baja Prioridad'
    else:
        return 'No Viable'

df['Clasificación'] = df.apply(lambda x: clasificar_prioridad(x['Beneficio (%)'], x['Indice Financiero Total (%)']), axis=1)

# 3. Configuración visual y cuadrantes
config = {
    'colors': {
        'Alta Prioridad': '#2E7D32',
        'Media Prioridad': '#FF8F00',
        'Baja Prioridad': '#1565C0',
        'No Viable': '#C62828'
    },
    'quadrants': [
        {'x1': 0, 'x2': 50, 'y1': 50, 'y2': 100, 'label': 'Alta Prioridad'},
        {'x1': 50, 'x2': 100, 'y1': 50, 'y2': 100, 'label': 'Media Prioridad'},
        {'x1': 0, 'x2': 50, 'y1': 0, 'y2': 50, 'label': 'Baja Prioridad'},
        {'x1': 50, 'x2': 100, 'y1': 0, 'y2': 50, 'label': 'No Viable'}
    ]
}

# 4. Construcción del gráfico
fig = go.Figure()

# Fondo de cuadrantes
for quad in config['quadrants']:
    fig.add_shape(
        type="rect",
        x0=quad['x1'], y0=quad['y1'],
        x1=quad['x2'], y1=quad['y2'],
        fillcolor=config['colors'][quad['label']],
        opacity=0.1,
        layer="below",
        line=dict(width=0)
    )
    fig.add_annotation(
        x=(quad['x1'] + quad['x2'])/2,
        y=(quad['y1'] + quad['y2'])/2,
        text=f"<b>{quad['label'].upper()}</b>",
        showarrow=False,
        font=dict(size=16, color=config['colors'][quad['label']]),
        xanchor="center",
        yanchor="middle"
    )

# Puntos de datos
for category, color in config['colors'].items():
    df_cat = df[df['Clasificación'] == category]
    fig.add_trace(go.Scatter(
        x=df_cat['Indice Financiero Total (%)'],
        y=df_cat['Beneficio (%)'],
        mode='markers',
        marker=dict(color=color, size=12, opacity=0.85, line=dict(width=1, color='white')),
        name=category,
        customdata=df_cat[['Iniciativa', 'Beneficio (%)', 'Indice Financiero Total (%)', 'Clasificación']],
        hovertemplate="<b>%{customdata[0]}</b><br>Clasificación: %{customdata[3]}<br>Beneficio: %{customdata[1]}%<br>Costo: %{customdata[2]}%<extra></extra>"
    ))

# Líneas divisorias
fig.add_shape(type="line", x0=50, y0=0, x1=50, y1=100, line=dict(color="black", width=2))
fig.add_shape(type="line", x0=0, y0=50, x1=100, y1=50, line=dict(color="black", width=2))

# 5. Layout cuadrado
fig.update_layout(
    title="<b>Matriz de Priorización de Iniciativas</b>",
    xaxis=dict(
        title='Costo Relativo (%)',
        range=[0, 100],
        dtick=10,
        showline=True,
        linewidth=2,
        constrain='domain',
        gridcolor="#e0e0e0"
    ),
    yaxis=dict(
        title='Beneficio (%)',
        range=[0, 100],
        dtick=10,
        showline=True,
        linewidth=2,
        constrain='domain',
        gridcolor="#e0e0e0"
    ),
    width=900,
    height=900,
    legend=dict(
        title='Clasificación',
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        x=0.5,
        xanchor="center"
    ),
    plot_bgcolor="white"
)

# 6. Mostrar gráfico
fig.show()

# 7. Para exportar si lo deseas
# fig.write_image("Matriz_Priorizacion_50-50.png", width=900, height=900, scale=2)
# fig.write_html("Matriz_Priorizacion_50-50.html")


**Progreso de tareas por depósito**

In [18]:
import pandas as pd

ruta = '/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Assesment-v.2.xlsx'

# Cargar la primera hoja del archivo
df = pd.read_excel(ruta)

# Mostrar las primeras filas para validar la estructura
print(df.head())




                   Id. de tarea  \
0  2J54rxFdqES3UlJW8DS_GmUAFFo3   
1  2gEt6J59Q0SZQO1NYaItOGUAEBSZ   
2  3WOzTvRZtEWXBA3N87ayyGUAA7YO   
3  3kM9djJ9U0KoesOXRp84oGUAFIAT   
4  4vYCsMIGsk6cnhmzkq47JmUAM3mV   

                                  Nombre de la tarea Nombre del depósito  \
0                          CM-Agente de adopción-001    Evaluación final   
1  SMC-Optimización de la Planeación de Capacidad...   Conceptualización   
2  OS-Plataforma para generación de reportes con ...           Discarded   
3  OS-Generacion de agente para detectar incumpli...   Conceptualización   
4                              AITU-OnboardAgent-001   Conceptualización   

      Progreso Prioridad                                         Asignado a  \
0     En curso     Media                  Diana Patricia Fernández Madrigal   
1     En curso     Media                  Diana Patricia Fernández Madrigal   
2  No iniciado     Media  Gerónimo Hazael Pérez Antonio (Shared);Diana P...   
3  No iniciado  

**Progreso por depósito**

In [19]:
# Agrupar por Nombre del depósito y Progreso y contar Id. de tarea
progreso_por_deposito = df.groupby(['Nombre del depósito', 'Progreso']).agg({'Id. de tarea':'count'}).reset_index()

# Renombrar columna
progreso_por_deposito = progreso_por_deposito.rename(columns={'Id. de tarea':'Total de tareas'})

# Mostrar tabla transformada
progreso_por_deposito


,Nombre del depósito,Progreso,Total de tareas
0,Conceptualización,En curso,6
1,Conceptualización,No iniciado,8
2,Discarded,En curso,7
3,Discarded,No iniciado,10
4,Evaluación final,En curso,24
5,Evaluación inicial,En curso,5
6,Layout,No iniciado,1
7,Stand By,En curso,3
8,Stand By,No iniciado,1


**Checklist completado promedio por depósito**

In [20]:
# Convertir completados a número y total
df[['Completados', 'Total']] = df['Elementos de la lista de comprobación completados'].str.split('/', expand=True).astype(float)

# Calcular porcentaje completado
df['% Completado'] = df['Completados'] / df['Total']

# Agrupar por depósito y sacar promedio
checklist_por_deposito = df.groupby('Nombre del depósito')['% Completado'].mean().reset_index()

checklist_por_deposito


,Nombre del depósito,% Completado
0,Conceptualización,0.500000
1,Discarded,0.448718
2,Evaluación final,0.805556
3,Evaluación inicial,0.500000
4,Layout,0.000000
5,Stand By,0.416667


In [21]:
import pandas as pd
import plotly.express as px

# Datos de porcentaje completado por depósito
data = {
    'Nombre del depósito': [
        'Conceptualización', 'Discarded', 'Evaluación final',
        'Evaluación inicial', 'Layout', 'Stand By'
    ],
    '% Completado': [0.50, 0.448718, 0.805556, 0.50, 0.0, 0.416667]
}

df_checklist = pd.DataFrame(data)

# Gráfico interactivo
fig = px.bar(
    df_checklist,
    x='Nombre del depósito',
    y='% Completado',
    text='% Completado',
    title='Porcentaje promedio de checklist completado por depósito',
    labels={'% Completado': 'Porcentaje Completado', 'Nombre del depósito': 'Depósito'},
    height=500
)

fig.update_traces(texttemplate='%{text:.1%}', textposition='outside')
fig.update_layout(yaxis_tickformat='.0%', uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()


In [22]:
import pandas as pd

# Datos originales
data = {
    'Nombre del depósito': [
        'Conceptualización', 'Discarded', 'Evaluación final',
        'Evaluación inicial', 'Layout', 'Stand By'
    ],
    '% Completado': [0.50, 0.448718, 0.805556, 0.50, 0.0, 0.416667]
}

df = pd.DataFrame(data)

# Filtrar depósitos que sí quieres considerar
filtro = ~df['Nombre del depósito'].isin(['Discarded', 'Layout', 'Stand By'])
df_filtrado = df[filtro]

# Calcular promedio
avance_general = df_filtrado['% Completado'].mean()

print(f"Avance general (sin descartados, layout y stand by): {avance_general:.2%}")


Avance general (sin descartados, layout y stand by): 60.19%


In [23]:
import plotly.graph_objects as go

# Usando el avance calculado antes
avance_general = avance_general  # Ya calculado en la celda anterior

fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = avance_general * 100,
    title = {'text': "Avance General (%)"},
    gauge = {
        'axis': {'range': [0, 100]},
        'bar': {'color': "green"},
        'steps': [
            {'range': [0, 50], 'color': "lightgray"},
            {'range': [50, 80], 'color': "lightblue"},
            {'range': [80, 100], 'color': "lightgreen"}
        ],
    }
))

fig.update_layout(height=400, width=500)
fig.show()


In [26]:
import pandas as pd

# Cargar ambos archivos
v1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Assesment.v1.xlsx')
v2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Assesment-v.2.xlsx')

# Normalizar nombre de la tarea (evitar errores por espacios o mayúsculas)
v1['Nombre de la tarea'] = v1['Nombre de la tarea'].str.strip().str.upper()
v2['Nombre de la tarea'] = v2['Nombre de la tarea'].str.strip().str.upper()

# Traer el Id. de tarea de v2 al v1 (como columna nueva)
v1_con_id = v1.merge(v2[['Id. de tarea', 'Nombre de la tarea']], on='Nombre de la tarea', how='left', suffixes=('', '_v2'))

# Verificar resultado
print(v1_con_id[['Nombre de la tarea', 'Id. de tarea', 'Id. de tarea_v2']].head())

# Si quieres puedes guardar el resultado:
# v1_con_id.to_excel('v1_con_id_consolidado.xlsx', index=False)


                                  Nombre de la tarea  Id. de tarea  \
0                             CCS-MANUALES PROSA-001             1   
1             VENTAS-CREACION DE CLIENTES NUEVOS-001             2   
2  SMC-ONBOARDING MEDIANTE AGENTE DE COPILOT AUTO...             3   
3  SMC-ONBOARDING MEDIANTE AGENTE DE COPILOT AUTO...             3   
4  SMC-ONBOARDING MEDIANTE AGENTE DE COPILOT AUTO...             3   

                Id. de tarea_v2  
0  ruErmnRSWE21CkdEmQrTOmUANsI3  
1  lKFDK_DvIEO6wHPMCsHgFmUAPKmE  
2  RZrwSeyeA0qEYPR2HXzqnmUAEibS  
3  Y4lHlkAUQEa5A5r1C-fIkmUAF90I  
4  cY1lr88mrUGXtBVWdXflx2UAFF3y  


In [27]:
# Si el Id. de tarea_v2 existe, úsalo; si no, usa el Id. de tarea de v1
v1_con_id['Id Consolidado'] = v1_con_id['Id. de tarea_v2'].combine_first(v1_con_id['Id. de tarea'])

# Revisar resultado
print(v1_con_id[['Nombre de la tarea', 'Id Consolidado']].head())

# Si quieres exportar:
# v1_con_id.to_excel('v1_con_id_consolidado_final.xlsx', index=False)


                                  Nombre de la tarea  \
0                             CCS-MANUALES PROSA-001   
1             VENTAS-CREACION DE CLIENTES NUEVOS-001   
2  SMC-ONBOARDING MEDIANTE AGENTE DE COPILOT AUTO...   
3  SMC-ONBOARDING MEDIANTE AGENTE DE COPILOT AUTO...   
4  SMC-ONBOARDING MEDIANTE AGENTE DE COPILOT AUTO...   

                 Id Consolidado  
0  ruErmnRSWE21CkdEmQrTOmUANsI3  
1  lKFDK_DvIEO6wHPMCsHgFmUAPKmE  
2  RZrwSeyeA0qEYPR2HXzqnmUAEibS  
3  Y4lHlkAUQEa5A5r1C-fIkmUAF90I  
4  cY1lr88mrUGXtBVWdXflx2UAFF3y  


In [32]:
v1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Assesment.v1.xlsx')
v2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Assesment-v.2.xlsx')


In [33]:
print(v1.columns)
print(v2.columns)


Index(['Id. de tarea', 'Nombre de la tarea', 'Nombre del depósito', 'Progreso',
       'Prioridad', 'Asignado a', 'Fecha de creación',
       'Elementos de la lista de comprobación completados', 'Completados',
       'Totales ', 'Pendiente', 'Promedio'],
      dtype='object')
Index(['Id. de tarea', 'Nombre de la tarea', 'Nombre del depósito', 'Progreso',
       'Prioridad', 'Asignado a', 'Creado por', 'Fecha de creación',
       'Fecha de inicio', 'Fecha de vencimiento', 'Es periódica',
       'Con retraso', 'Fecha de finalización', 'Completado por',
       'Elementos de la lista de comprobación completados',
       'Elementos de la lista de comprobación', 'Etiquetas', 'Descripción'],
      dtype='object')


In [37]:
# Asegurar que la columna es numérica
v1['Elementos de la lista de comprobación completados'] = pd.to_numeric(v1['Elementos de la lista de comprobación completados'], errors='coerce').fillna(0)
v2['Elementos de la lista de comprobación completados'] = pd.to_numeric(v2['Elementos de la lista de comprobación completados'], errors='coerce').fillna(0)


**Normalizar ambas tablas**


In [40]:
import pandas as pd

# Cargar la versión 1
v1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Assesment.v1.xlsx')
v1['Completados'] = pd.to_numeric(v1['Completados'], errors='coerce').fillna(0)

# Cargar la versión 2
v2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Assesment-v.2.xlsx')
# Extraer número antes de la barra y convertir a numérico
v2['Completados'] = v2['Elementos de la lista de comprobación completados'].str.extract(r'(\d+)').astype(float).fillna(0)


**Calcular el total de checklists completados y total de tareas**

In [41]:
# Para v1
avance_v1 = pd.DataFrame({
    'Fecha de Corte': ['30-Jun'],
    'Total Checklists Completados': [v1['Completados'].sum()],
    'Total Tareas': [len(v1)],
    'Promedio Checklist/Tarea': [v1['Completados'].sum() / len(v1)]
})

# Para v2
avance_v2 = pd.DataFrame({
    'Fecha de Corte': ['17-Jul'],
    'Total Checklists Completados': [v2['Completados'].sum()],
    'Total Tareas': [len(v2)],
    'Promedio Checklist/Tarea': [v2['Completados'].sum() / len(v2)]
})

# Unir ambos
resumen = pd.concat([avance_v1, avance_v2]).reset_index(drop=True)


**Evolución avance total **

In [42]:
import plotly.express as px

fig = px.bar(resumen, x='Fecha de Corte', y='Total Checklists Completados',
             text='Total Checklists Completados',
             title='Evolución Total de Checklists Completados (30-Jun vs 17-Jul)')
fig.update_traces(textposition='outside')
fig.update_layout(yaxis_title='Total Checklists Completados',
                  xaxis_title='Fecha de Corte',
                  uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()


**Nombre del depósito**

In [43]:
v1_grouped = v1.groupby('Nombre del depósito')['Completados'].sum().reset_index().assign(Fecha='30-Jun')
v2_grouped = v2.groupby('Nombre del depósito')['Completados'].sum().reset_index().assign(Fecha='17-Jul')

comparativo = pd.concat([v1_grouped, v2_grouped])

fig = px.bar(comparativo, x='Nombre del depósito', y='Completados',
             color='Fecha', barmode='group',
             title='Checklists Completados por Depósito (30-Jun vs 17-Jul)')
fig.show()


In [44]:
import pandas as pd
import plotly.graph_objects as go

# Cargar v1
v1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Assesment.v1.xlsx')
v1['Completados'] = pd.to_numeric(v1['Completados'], errors='coerce').fillna(0)

# Cargar v2
v2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Onesec-AITU/Análisis de datos/Assesment-v.2.xlsx')
v2['Completados'] = v2['Elementos de la lista de comprobación completados'].str.extract(r'(\d+)').astype(float).fillna(0)

# Datos resumen
data_resumen = pd.DataFrame({
    'Fecha de Corte': ['30-Jun', '17-Jul'],
    'Total Checklists Completados': [v1['Completados'].sum(), v2['Completados'].sum()],
    'Total Tareas Registradas': [len(v1), len(v2)],
    'Avance Promedio Checklist por Tarea': [
        round(v1['Completados'].sum() / len(v1), 2),
        round(v2['Completados'].sum() / len(v2), 2)
    ]
})

# Gráfico combinado
fig = go.Figure()

# Barras de avance total
fig.add_trace(go.Bar(
    x=data_resumen['Fecha de Corte'],
    y=data_resumen['Total Checklists Completados'],
    name='Checklists Completados',
    marker_color='#2E86AB',
    text=data_resumen['Total Checklists Completados'],
    textposition='outside'
))

# Línea de tareas registradas
fig.add_trace(go.Scatter(
    x=data_resumen['Fecha de Corte'],
    y=data_resumen['Total Tareas Registradas'],
    name='Total Tareas Registradas',
    mode='lines+markers+text',
    text=data_resumen['Total Tareas Registradas'],
    textposition='bottom center',
    line=dict(color='#E15759', width=3, dash='dot')
))

# Línea de avance promedio
fig.add_trace(go.Scatter(
    x=data_resumen['Fecha de Corte'],
    y=data_resumen['Avance Promedio Checklist por Tarea'],
    name='Promedio Checklist/Tarea',
    mode='lines+markers+text',
    text=data_resumen['Avance Promedio Checklist por Tarea'],
    textposition='top center',
    line=dict(color='#76B041', width=3)
))

# Ajustes de layout
fig.update_layout(
    title='<b>Evolución del Avance Global de Iniciativas</b>',
    xaxis_title='Fecha de Corte',
    yaxis_title='Totales',
    legend_title='Indicadores',
    template='plotly_white',
    bargap=0.4,
    font=dict(size=12),
    height=500
)

fig.show()


In [46]:
import plotly.graph_objects as go

# Datos
base = 150
avance = 68
total = 218
porcentaje_avance = (avance / base) * 100

# Gráfico tipo indicador + número
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode="gauge+number+delta",
    value=porcentaje_avance,
    delta={'reference': 0, 'increasing': {'color': "green"}},
    title={'text': "Incremento % de Checklists Completados<br><span style='font-size:0.8em;color:gray'>Del 30-Jun al 17-Jul</span>"},
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': "green"},
        'steps': [
            {'range': [0, 50], 'color': "#d3f9d8"},
            {'range': [50, 100], 'color': "#b9fbc0"}
        ],
    }
))

fig.update_layout(height=400, width=500)
fig.show()


En solo dos semanas, logramos un 45% de incremento en la ejecución de checklists, reflejando la aceleración y el compromiso del equipo hacia el objetivo

**Stacked Bar de Checklists por Fecha**

In [47]:
import plotly.express as px
import pandas as pd

# Simulación de datos totales (puedes ajustar)
data = pd.DataFrame({
    'Fecha': ['30-Jun', '17-Jul'],
    'Checklists Completados': [150, 218],
    'Pendientes': [228 - 150, 228 - 218]
})

fig = px.bar(
    data,
    x='Fecha',
    y=['Checklists Completados', 'Pendientes'],
    title='Estatus de Checklists por Fecha de Corte',
    text_auto=True
)
fig.update_layout(barmode='stack', yaxis_title='Total de Checklists', xaxis_title='Fecha de Corte')
fig.show()


**Pie Chart de Distribución al 17-Jul**

In [48]:
data = pd.DataFrame({
    'Estatus': ['Completados', 'Pendientes'],
    'Cantidad': [218, 228 - 218]
})

fig = px.pie(
    data,
    values='Cantidad',
    names='Estatus',
    title='Distribución de Checklists al 17-Jul',
    hole=0.4
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


**Waterfall de Acumulación de Avance (Estilo Ejecutivo)**

In [49]:
import plotly.graph_objects as go

fig = go.Figure(go.Waterfall(
    name = "Avance",
    orientation = "v",
    measure = ["absolute", "relative", "total"],
    x = ["Checklists al 30-Jun", "Avance 1-17 Jul", "Total al 17-Jul"],
    textposition = "outside",
    text = ["150", "+68", "218"],
    y = [150, 68, 218],
    connector = {"line":{"color":"rgb(63, 63, 63)"}}
))

fig.update_layout(
        title = "Impacto del Avance Acumulado de Checklists",
        showlegend = False,
        yaxis_title = "Total de Checklists"
)

fig.show()
